## All you need is love… And a pet!

<img src="img/dataset-cover.jpg" width="920">

Here we are going to build a classifier to predict whether an animal from an animal shelter will be adopted or not (aac_intakes_outcomes.csv, available at: https://www.kaggle.com/aaronschlegel/austin-animal-center-shelter-intakes-and-outcomes/version/1#aac_intakes_outcomes.csv). You will be working with the following features:

1. *animal_type:* Type of animal. May be one of 'cat', 'dog', 'bird', etc.
2. *intake_year:* Year of intake
3. *intake_condition:* The intake condition of the animal. Can be one of 'normal', 'injured', 'sick', etc.
4. *intake_number:* The intake number denoting the number of occurrences the animal has been brought into the shelter. Values higher than 1 indicate the animal has been taken into the shelter on more than one occasion.
5. *intake_type:* The type of intake, for example, 'stray', 'owner surrender', etc.
6. *sex_upon_intake:* The gender of the animal and if it has been spayed or neutered at the time of intake
7. *age_upon\_intake_(years):* The age of the animal upon intake represented in years
8. *time_in_shelter_days:* Numeric value denoting the number of days the animal remained at the shelter from intake to outcome.
9. *sex_upon_outcome:* The gender of the animal and if it has been spayed or neutered at time of outcome
10. *age_upon\_outcome_(years):* The age of the animal upon outcome represented in years
11. *outcome_type:* The outcome type. Can be one of ‘adopted’, ‘transferred’, etc.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp
from itertools import combinations 
import ast
from sklearn.linear_model import LogisticRegression
import seaborn as sn
%matplotlib inline

data_folder = './data/'

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### A) Load the dataset and convert categorical features to a suitable numerical representation (use dummy-variable encoding). 
- Split the data into a training set (80%) and a test set (20%). Pair each feature vector with the corresponding label, i.e., whether the outcome_type is adoption or not. 
- Standardize the values of each feature in the data to have mean 0 and variance 1.

The use of external libraries is not permitted in part A, except for numpy and pandas. 
You can drop entries with missing values.

In [3]:
columns = ['animal_type', 'intake_year', 'intake_condition', 'intake_number', 'intake_type', 'sex_upon_intake', \
          'age_upon_intake_(years)', 'time_in_shelter_days', 'sex_upon_outcome', 'age_upon_outcome_(years)', \
          'outcome_type']
original_data = pd.read_csv(data_folder+'aac_intakes_outcomes.csv', usecols=columns)

original_data.dropna()
y = original_data['outcome_type']
y = np.where(1, y=='Adoption', 0) #3shan teb2a 3aref myn ehh!!!!
y = pd.DataFrame(y)


X = original_data.copy()
X.drop('outcome_type', axis=1, inplace=True) #mohemma awy inplace=True!!!!!
X = pd.get_dummies(X)

np.random.seed(42)
shuffled_indices = np.random.permutation(len(X))
X_shuffled = X.iloc[shuffled_indices, :]#3shan te3raf te5tar elshuffled indices!!!
y_shuffled = y.iloc[shuffled_indices] #mafysh clumns la2en howa aslan column wa7ed!!!

split_index = int(0.8*len(X_shuffled))#3shan te3raf hato2af fein w hatebda2 fein!!!
X_train = X_shuffled.iloc[:split_index, :]
y_train = y_shuffled.iloc[:split_index]
y_train = np.array(y_train).ravel()

X_train = (X_train-np.mean(X_train, axis = 0))/np.std(X_train, axis = 0)

X_test = X_shuffled.iloc[split_index:, :]
y_test = y_shuffled.iloc[split_index:]
y_test = np.array(y_test).ravel()
print(y_test.shape)

X_test = (X_test-np.mean(X_test, axis = 0))/np.std(X_test, axis = 0)

(15935,)


### B) Train a logistic regression classifier on your training set. Logistic regression returns probabilities as predictions, so in order to arrive at a binary prediction, you need to put a threshold on the predicted probabilities. 
- For the decision threshold of 0.5, present the performance of your classifier on the test set by displaying the confusion matrix. Based on the confusion matrix, manually calculate accuracy, precision, recall, and F1-score with respect to the positive and the negative class. 

[[7577 1574]
 [1225 5559]]


In [4]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
y_test_pred = log_reg.predict(X_test)
confusion_matrix_result = confusion_matrix(y_test, y_test_pred)
print(confusion_matrix_result) #tn, fp, fn, tp !!!
#cm = confusion_matrix(y_test, predictions, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix_result)
disp.plot()
plt.show()

In [5]:
accuracy = (confusion_matrix_result[0, 0]+confusion_matrix_result[1, 1])/confusion_matrix_result.sum()
precision = confusion_matrix_result[1, 1]/(confusion_matrix_result[0, 1]+confusion_matrix_result[1, 1])
recall = confusion_matrix_result[1, 1]/(confusion_matrix_result[1, 0]+confusion_matrix_result[1, 1])
F1_score = (2*precision*recall/(precision+recall))
print(accuracy, precision, recall, F1_score)

0.8243489174772514 0.7793354829664938 0.8194280660377359 0.7988790687648201


### C) Vary the value of the threshold in the range from 0 to 1 and visualize the value of accuracy, precision, recall, and F1-score (with respect to both classes) as a function of the threshold.

/tmp/ipykernel_161151/1702104069.py:19: RuntimeWarning: invalid value encountered in scalar divide
  precision = confusion_matrix_result[1, 1]/(confusion_matrix_result[0, 1]+confusion_matrix_result[1, 1])


In [7]:
threshold = np.linspace(0, 1, 100)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
acc_vect = []
prec_vect = []
recall_vect=[]
F1_score_vect = []
for t in threshold:
    log_reg = LogisticRegression()
    log_reg.fit(X_train, y_train)

    y_test_pred_prob = log_reg.predict_proba(X_test) 
    #print(y_test_pred_prob)
    y_test_pred = (np.amax(y_test_pred_prob, axis=1) > t).astype(int)
    y_test_pred = pd.DataFrame(y_test_pred)
    #print(y_test_pred)
    confusion_matrix_result = confusion_matrix(y_test, y_test_pred)
    accuracy = (confusion_matrix_result[0, 0]+confusion_matrix_result[1, 1])/confusion_matrix_result.sum()
    acc_vect.append(accuracy)
    precision = confusion_matrix_result[1, 1]/(confusion_matrix_result[0, 1]+confusion_matrix_result[1, 1])
    prec_vect.append(precision)
    recall = confusion_matrix_result[1, 1]/(confusion_matrix_result[1, 0]+confusion_matrix_result[1, 1])
    recall_vect.append(recall)
    F1_score = (2*precision*recall/(precision+recall))
    F1_score_vect.append(F1_score)

fig, ax = plt.subplots()
ax.plot(threshold, acc_vect)
ax.plot(threshold, prec_vect)
ax.plot(threshold, recall_vect)
ax.plot(threshold, F1_score_vect)
ax.legend(['Accuracy', 'Precision', 'Recall', 'F1-score'])
ax.set_xlabel('Threshold')
ax.set_ylabel('Score')
ax.set_title('Accuracy, Precision, Recall, F1-score as a function of the classification threshold')

ax.grid()
plt.tight_layout()
plt.show()

### D) Plot in a bar chart the coefficients of the logistic regression sorted by their contribution to the prediction.

In [ ]:
#PLT.BARH 3SHAN TEDDY HORIZONTAL BARS!!!


## Question 1: Which of the following metrics is most suitable when you are dealing with unbalanced classes?

- a) F1 Score
- b) Recall
- c) Precision
- d) Accuracy

In [ ]:
a

## Question 2: You are working on a binary classification problem. You trained a model on a training dataset and got the following confusion matrix on the test dataset. What is true about the evaluation metrics (rounded to the second decimal point):

|            | Pred = NO|Pred=YES|
|------------|----------|--------|
| Actual NO  |    50    |   10   |
| Actual YES |    5     |   100  |

- a) Accuracy is 0.95
- b) Accuracy is 0.85
- c) False positive rate is 0.95
- d) True positive rate is 0.95

In [ ]:
d